<a href="https://colab.research.google.com/github/sugiyama404/ReinfoceLearningForTrading/blob/main/PPO/ppo_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from datetime import datetime
import random
import copy
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

import math

from google.colab import drive

from tensorflow.keras.utils import Progbar

from concurrent.futures import ThreadPoolExecutor

from dataclasses import dataclass, field
from typing import List

mode = 'train'
name = 'ppo'
level = 1
if level == 2:
    name += name + 'lv2'

drive.mount('/content/drive/')
nov_dir = 'Colab Notebooks/dataset/reinforcement_learning/'
nov_path = '/content/drive/My Drive/' + nov_dir + f'sp500_{mode}.csv'

exp_dir = 'Colab Notebooks/workspace/export/'
mdl_dir = '/content/drive/My Drive/' + exp_dir + 'models'
csv_path = '/content/drive/My Drive/' + exp_dir + f'csv_data/{name}_{mode}.csv'

df = pd.read_csv(nov_path)
df['Date'] = pd.to_datetime(df['Date'], format = '%Y-%m-%d')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
class Environment:
    def __init__(self, df, initial_money=100000, mode = 'test', commission = 0):

        self.df = df.dropna().reset_index()

        self.df_total_steps  = len(self.df)-1
        self.initial_money   = initial_money
        self.mode            = mode
        self.commission      = commission
        self.trade_time      = None
        self.trade_win       = None
        self.brfore_buy_cash = None
        self.action_space    = np.array([0, 1, 2]) # buy,hold,sell
        self.hold_a_position = None
        self.now_price       = None
        self.cash_in_hand    = None
        self.sell_price      = None
        self.buy_price       = None

        self.reset()
        
    def reset(self):

        self.trade_time      = 0
        self.trade_win       = 0
        self.brfore_buy_cash = 0
        self.end_step        = self.df_total_steps
        self.now_step        = 0
        self.hold_a_position = 0.0
        self.now_price       = self.df.loc[self.now_step, 'SP500']
        self.cash_in_hand    = self.initial_money
        self.sell_price      = 0
        self.buy_price       = 0

        return self._get_now_state()

    def step(self, action):

        self.now_step += 1
        self.now_price = self.df.loc[self.now_step, 'SP500']
 
        done = (self.end_step == self.now_step)

        self.sell_price = 0
        self._trade(action,done)
        reward = 0
        if (self.sell_price > 0) and (self.buy_price > 0) and ((self.sell_price - self.buy_price) != 0):
            reward = (self.sell_price - self.buy_price) / self.buy_price
            self.buy_price = 0
        cur_revenue = self._get_revenue()
 
        info = { 'cur_revenue' : cur_revenue , 'trade_time' : self.trade_time, 'trade_win' : self.trade_win }

        return self._get_now_state(), reward, done, info

    def _get_now_state(self):
        state = np.empty(3)
        state[0] = self.hold_a_position
        state[1] = self.now_price
        state[2] = self.cash_in_hand
        return state

    def _get_revenue(self): 
        return self.hold_a_position * self.now_price + self.cash_in_hand

    def _trade(self, action,lastorder = False):
        if lastorder:
            if self.hold_a_position != 0:
                self.cash_in_hand += self.now_price * self.hold_a_position
                self.hold_a_position = 0
                self.trade_time += 1
                if self.cash_in_hand > self.brfore_buy_cash:
                    self.trade_win += 1
        else:
            if self.action_space[0] == action: # buy
                if self.hold_a_position == 0:
                    buy_flag = True
                    self.brfore_buy_cash = copy.copy(self.cash_in_hand)
                    while buy_flag:
                        if self.cash_in_hand > self.now_price:
                            self.hold_a_position += 1
                            self.buy_price += self.now_price
                            self.cash_in_hand -= self.now_price + self.commission * self.now_price
                        else:
                            buy_flag = False
            if self.action_space[2] == action: # sell
                if self.hold_a_position != 0:
                    self.sell_price += self.now_price * self.hold_a_position
                    self.cash_in_hand += self.now_price * self.hold_a_position - self.commission * self.now_price * self.hold_a_position
                    self.hold_a_position = 0
                    self.trade_time += 1
                    if self.cash_in_hand > self.brfore_buy_cash:
                        self.trade_win += 1

In [3]:
class ParameterServer:
    def __init__(self):

        opt = Adam(learning_rate=0.0001, epsilon=0.001)

        common = input_ = keras.layers.Input(shape=3)
        common = keras.layers.Dense(128, activation="relu")(common)

        actor = keras.layers.Dense(3, activation="softmax")(common)
        critic = keras.layers.Dense(1, activation="linear")(common)

        model = keras.Model(input_, [actor, critic])
        model.compile(optimizer=opt)
        model.summary()
        self.model = model

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [4]:
class Actor:
    def __init__(self, brain):
        self.model = brain.model
        self.brain = brain
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.r = 0.995

    def policynetwork(self, state):
        if np.random.rand() <= self.epsilon:
            self._reduce_epsilon()
            return np.random.choice(3)
        act_p, _ = self.model(state.reshape((1,-1)))
        self._reduce_epsilon()
        return np.random.choice(3, p=act_p[0].numpy())

    def load(self, name):
        self.brain.load(name)

    def save(self, name):
        self.brain.save(name)

    def _reduce_epsilon(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.r

In [5]:
class Critic:
    def __init__(self,model):
        self.model = model
        self.gamma = 0.2
        self.beta_1 = 0.1
        self.beta_2 = 0.1

    def valuenetwork(self, experiences):

        state_arr  = np.asarray([e["state"] for e in experiences])
        action_arr = np.asarray([e["action"] for e in experiences])
        advantage  = np.asarray([e["advantage"] for e in experiences]).reshape((-1, 1))
        prob_arr   = np.asarray([e["prob"] for e in experiences])

        advantage = (advantage - np.mean(advantage)) / (np.std(advantage) + 1e-8)
        onehot_actions = tf.one_hot(action_arr, 3).numpy()

        old_policy = tf.reduce_sum(onehot_actions * prob_arr, axis=1, keepdims=True)

        with tf.GradientTape() as tape:
            act_p, v = self.model(state_arr, training=True)

            #advantage = advantage - tf.stop_gradient(v)
            advantage = advantage - v

            new_policy = tf.reduce_sum(onehot_actions * act_p, axis=1, keepdims=True)
            log_new_policy = tf.math.log(tf.clip_by_value(new_policy, 1e-10, 1.0))

            policy_ration = new_policy / old_policy

            losses_clip  = self._losses_clip(advantage, policy_ration)
            losses_value = tf.stop_gradient(self._losses_value(advantage))
            entropy      = tf.reduce_sum(log_new_policy * new_policy, axis=1, keepdims=True)

            total_loss   = losses_clip - self.beta_1 * losses_value - self.beta_2 * entropy
            loss         = tf.reduce_mean(total_loss)


        gradients = tape.gradient(loss, self.model.trainable_variables)

        self.model.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))

    def _losses_value(self,advantage):
        return (advantage)**2

    def _losses_clip(self, advantage, policy_ration):

        r_clip = tf.clip_by_value(policy_ration, 1 - self.gamma, 1 + self.gamma)

        loss_unclipped = policy_ration * advantage
        loss_clipped = r_clip * advantage

        return tf.minimum(loss_clipped, loss_clipped)

In [6]:
@dataclass
class ExperiencesMemory:
    state : List[List[float]] = field(default_factory=list)
    action : List[int] = field(default_factory=list)
    reward : List[float] = field(default_factory=list)
    next_state : List[List[int]] = field(default_factory=list)
    done : List[bool] = field(default_factory=list)
    prob_bol : List[bool] = field(default_factory=list)
    advantage : List[float] = field(default_factory=list)
    model : str = None
    gae_lambda: float = 0.9
    gamma: float = 0.9

    def append_experiences(self, state, action, reward, next_state, done):
        self.state.append(state)
        self.action.append(action)
        self.reward.append(reward)
        self.next_state.append(next_state)
        self.done.append(done)

    def random_experiences(self, batch_size):
        max_size = len(self.state) - 1
        self._make_advantage()
        batch_num = self._random_num(1, max_size, batch_size)
        experiences = []
        for i in batch_num:
            prob = self._probability(self.state[i])
            prob = prob.numpy()
            prob = prob[0]
            experiences.append({"state": self.state[i], "action": self.action[i], 
                                "reward": self.reward[i], "next_state": self.next_state[i], 
                                "done": self.done[i], "prob": prob, "advantage": self.advantage[i]})
        return experiences

    def _random_num(self, min_num, max_num, batch_size):
        arr = []
        while len(arr) < batch_size:
            n = random.randint(min_num, max_num)
            if not n in arr:
                arr.append(n)
        return arr

    def _make_advantage(self):
        state = np.asarray([e for e in self.state])
        next_state = np.asarray([e for e in self.next_state])
        v     = self._evaluation_value(state)
        v_old = self._evaluation_value(next_state)
        v     =v.numpy()
        v_old =v_old.numpy()

        for i in range(len(self.reward)):
            gae = 0
            t = 0
            for j in range(i, len(self.reward)):
                delta = self.reward[j] + self.gamma * v_old[j][0] - v[j][0]
                gae += ((self.gae_lambda * self.gamma) ** t) * delta
                t += 1
            self.advantage.append(gae)

    def _probability(self, state):
        p, _ = self.model(state.reshape((1,-1)))
        return p

    def _evaluation_value(self, state):
        _, v = self.model(state, training=True)
        return v

In [7]:
class Main:
    def __init__(self, env, actor, critic, num, mdl_dir, name, batch_size = 128, episodes_times = 1000, mode = 'test'):
        self.env = env
        self.actor = actor
        self.critic = critic
        self.num = str(num)
        self.mdl_dir = mdl_dir
        self.scaler = self._standard_scaler(self.env)
        self.episodes_times = episodes_times
        self.batch_size = batch_size
        self.mode = mode
        self.name = name

        with open(csv_path, 'w') as f:
            row = 'FixedProfit,TradeTimes,TradeWin'
            print(row, file=f)

        if self.mode == 'test':
            self._load()
            self.actor.epsilon = 0.01

    def play_game(self):

        try:
            for episode in range(self.episodes_times):

                if (episode % 10 == 0):
                    metrics_names = ['FixedProfit','TradeTimes','TradeWin']
                    if (int(str(self.episodes_times)[:-1])*10 == episode):
                        pb_i = Progbar(int(str(self.episodes_times)[-1]), stateful_metrics=metrics_names)
                    else:
                        pb_i = Progbar(10, stateful_metrics=metrics_names)
                    p_mean,trade_time,win_time = np.array([]),np.array([]),np.array([])

                state = self.env.reset()
                state = self.scaler.transform([state])
                state = state.flatten()
                done = False
                memory = ExperiencesMemory(model = self.actor.model)
        
                while not done:
                    
                    action     = self.actor.policynetwork(state)
                    next_state, reward, done, info = self.env.step(action)
                    next_state = self.scaler.transform([next_state])
                    next_state = next_state.flatten()
                    reward = self._reward_clipping(reward)

                    if self.mode == 'train':
                        memory.append_experiences(state, action, reward, next_state, done)

                    state = next_state

                if self.mode == 'train':
                    experiences = memory.random_experiences(self.batch_size)
                    self.critic.valuenetwork(experiences)

                p_mean,trade_time,win_time = np.append(p_mean,info['cur_revenue']),np.append(trade_time,info['trade_time']),np.append(win_time,info['trade_win'])
                values=[('FixedProfit',int(np.mean(p_mean))), ('TradeTimes',int(np.mean(trade_time))), ('TradeWin',int(np.mean(win_time)))]
                pb_i.add(1, values=values)
                with open(csv_path, 'a') as f:
                    row = str(info['cur_revenue']) + ',' + str(info['trade_time']) + ',' + str(info['trade_win'])
                    print(row, file=f)

        except Exception as e:
            print(e)

        if self.mode == 'train':
            self._save()

    def _standard_scaler(self, env):
        states = []
        for _ in range(env.df_total_steps):
            action = np.random.choice(env.action_space)
            state, reward, done, info = env.step(action)
            states.append(state)
            if done:
                break
        
        scaler = StandardScaler()
        scaler.fit(states)
        return scaler

    def _reward_clipping(self, val):
        result = 1 if val > 0 else 0 if val == 0 else -1
        return result

    def _load(self):
        with open('{}/{}_{}.pkl'.format(self.mdl_dir, self.name, self.num), 'rb') as f:
            self.scaler = pickle.load(f)
        self.actor.load('{}/{}.h5'.format(self.mdl_dir, self.name))

    def _save(self):
        self.actor.save('{}/{}.h5'.format(self.mdl_dir, self.name))
        with open('{}/{}_{}.pkl'.format(self.mdl_dir, self.name, self.num), 'wb') as f:
            pickle.dump(self.scaler, f)

In [8]:
initial_money=1000000
episodes_times = 50
commission = 0 if level == 1 else 0.002
batch_size = 128

brain = ParameterServer()

thread_num = 4
envs = []
for i in range(thread_num):
    env = Environment(df, initial_money=initial_money,mode = mode, commission = commission)
    model = brain.model
    actor = Actor(brain)
    critic = Critic(model)
    main = Main(env, actor, critic, i, mdl_dir, name, batch_size, episodes_times, mode)
    envs.append(main)

datas = []
with ThreadPoolExecutor(max_workers=thread_num) as executor:
    for env in envs:
        job = lambda: env.play_game()
        datas.append(executor.submit(job))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          512         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 3)            387         dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 1)            129         dense[0][0]                      
Total params: 1,028
Trainable params: 1,028
Non-trainable params: 0
__________________________